# Timeout-Based Disconnect & Smart Port Management

## New Improvements Added

### 🚀 **Smart Disconnect with Timeouts**
- **Non-blocking cleanup**: Disconnect won't hang waiting for browser tabs
- **Timeout-based process termination**: Max 3 seconds per port cleanup
- **Graceful degradation**: Continues even if some ports can't be freed
- **Visual feedback**: Shows real-time progress during disconnect

### 🔧 **Intelligent Port Management**
- **Smart port selection**: Analyzes why ports are busy
- **Alternative port finding**: Automatically uses next available port
- **Browser tab detection**: Identifies when ports are held by open notebook tabs
- **Cleanup reporting**: Shows which ports were freed vs still in use

### 📊 **Enhanced User Feedback**
- **Progress indicators**: Disconnect button shows "Disconnecting..." status
- **Port status messages**: Clear explanation of port conflicts
- **Cleanup summaries**: Reports how many ports were successfully freed
- **Helpful hints**: Suggests closing browser tabs to fully free ports

In [ ]:
# Example: Timeout-based port cleanup function
import psutil
import subprocess
import time
import signal

def kill_processes_on_port_with_timeout(port, timeout=3):
    """Kill processes on a port with timeout to avoid hanging."""
    killed_processes = []
    
    try:
        # Set a timeout to avoid hanging indefinitely
        start_time = time.time()
        
        for proc in psutil.process_iter(['pid', 'name', 'connections']):
            # Check if we've exceeded our timeout
            if time.time() - start_time > timeout:
                print(f"⏱️  Timeout reached ({timeout}s) - stopping cleanup")
                break
                
            try:
                connections = proc.info['connections']
                if connections:
                    for conn in connections:
                        if hasattr(conn, 'laddr') and conn.laddr and conn.laddr.port == port:
                            print(f"🔍 Found process using port {port}: PID {proc.info['pid']} ({proc.info['name']})")
                            
                            # Try graceful termination first
                            proc.terminate()
                            try:
                                proc.wait(timeout=1)  # Wait 1 second for graceful shutdown
                                print(f"✅ Process terminated gracefully")
                            except psutil.TimeoutExpired:
                                proc.kill()  # Force kill if needed
                                print(f"⚡ Process force-killed")
                            
                            killed_processes.append(f"PID {proc.info['pid']} ({proc.info['name']})")
                            break
            except (psutil.NoSuchProcess, psutil.AccessDenied):
                continue
                
    except Exception as e:
        print(f"❌ Error during cleanup: {e}")
    
    return killed_processes

# Test the function (simulation)
print("🧪 Testing timeout-based port cleanup:")
print("✅ Function defined - ready for non-blocking port cleanup")
print("⏱️  Max timeout: 3 seconds per port")
print("🔄 Graceful termination → Force kill if needed")
print("📊 Returns list of cleaned processes")

# Jupyter Session Manager Improvements

This notebook demonstrates the improvements made to handle port conflicts and provide better session management.

## Key Improvements

### 1. Enhanced Port Management
- **Better port checking**: Uses multiple methods to detect if a port is in use
- **Process detection**: Identifies processes using specific ports
- **Automatic cleanup**: Kills lingering SSH tunnels and processes
- **Dynamic port assignment**: Finds alternative ports if needed

### 2. Session Output Capture
- **Copy to notebook**: Export session logs as Jupyter notebooks
- **Real-time monitoring**: Live display of session progress
- **Formatted output**: Timestamped and categorized log messages

### 3. Improved Disconnect Logic
- **Clean disconnection**: Properly terminates SSH tunnels
- **Resource cleanup**: Frees ports and closes connections
- **Session preservation**: Maintains SSH connection for server switching

## Port Management Functions

The improved port checking includes multiple validation methods:

In [ ]:
import socket
import psutil

def is_port_free(port, host='localhost'):
    """Enhanced port checking with multiple validation methods."""
    
    # Method 1: Socket bind test
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.bind((host, port))
            s.listen(1)
    except OSError:
        print(f"Port {port} is in use (socket bind test)")
        return False
    
    # Method 2: Process connection check
    try:
        for conn in psutil.net_connections():
            if conn.laddr.port == port and conn.status in ['LISTEN', 'ESTABLISHED']:
                print(f"Port {port} is in use (process connection check)")
                return False
    except (psutil.AccessDenied, AttributeError):
        pass
    
    # Method 3: Connection attempt
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.settimeout(1)
        result = s.connect_ex((host, port))
        if result == 0:
            print(f"Port {port} is in use (connection test)")
            return False
    
    print(f"Port {port} is free")
    return True

# Test port checking
test_ports = [8888, 8889, 8890]
for port in test_ports:
    status = "FREE" if is_port_free(port) else "IN USE"
    print(f"Port {port}: {status}")

## Process Cleanup Functions

Functions to identify and terminate processes using specific ports:

In [ ]:
def find_processes_on_port(port):
    """Find all processes using a specific port."""
    processes = []
    
    try:
        for proc in psutil.process_iter(['pid', 'name', 'connections']):
            try:
                connections = proc.info['connections']
                if connections:
                    for conn in connections:
                        if hasattr(conn, 'laddr') and conn.laddr and conn.laddr.port == port:
                            processes.append({
                                'pid': proc.info['pid'],
                                'name': proc.info['name'],
                                'status': conn.status
                            })
                            break
            except (psutil.NoSuchProcess, psutil.AccessDenied):
                continue
    except Exception as e:
        print(f"Error finding processes on port {port}: {e}")
    
    return processes

# Check what's using common Jupyter ports
for port in range(8888, 8893):
    procs = find_processes_on_port(port)
    if procs:
        print(f"Port {port} used by: {procs}")
    else:
        print(f"Port {port} is free")

## Session Output Management

The session manager now captures all output in a structured format:

In [ ]:
import json
from datetime import datetime

# Example session output format
sample_session_log = [
    {
        "timestamp": "14:23:15",
        "message": "Starting Jupyter Notebook session...",
        "type": "info"
    },
    {
        "timestamp": "14:23:16",
        "message": "Target server: gpu01",
        "type": "info"
    },
    {
        "timestamp": "14:23:17",
        "message": "Port 8888 is in use, attempting cleanup...",
        "type": "warning"
    },
    {
        "timestamp": "14:23:18",
        "message": "Killed processes: PID 1234 (ssh)",
        "type": "warning"
    },
    {
        "timestamp": "14:23:19",
        "message": "Using local port: 8889",
        "type": "success"
    },
    {
        "timestamp": "14:23:25",
        "message": "Jupyter Notebook URL: http://localhost:8889/?token=abc123",
        "type": "success"
    }
]

# Convert to notebook format
def create_session_notebook(session_log, title="Jupyter Session Log"):
    """Convert session log to notebook format."""
    
    # Format log as text
    log_text = "\n".join([
        f"[{entry['timestamp']}] [{entry['type'].upper()}] {entry['message']}"
        for entry in session_log
    ])
    
    notebook = {
        "cells": [
            {
                "cell_type": "markdown",
                "metadata": {},
                "source": [
                    f"# {title}\n",
                    f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n",
                    "\n",
                    "## Session Log\n",
                    "```\n",
                    log_text,
                    "\n```"
                ]
            },
            {
                "cell_type": "code",
                "execution_count": None,
                "metadata": {},
                "outputs": [],
                "source": [
                    "# Environment verification\n",
                    "import torch\n",
                    "import sys\n",
                    "\n",
                    "print(f'PyTorch version: {torch.__version__}')\n",
                    "print(f'CUDA available: {torch.cuda.is_available()}')\n",
                    "print(f'Python version: {sys.version}')\n"
                ]
            }
        ],
        "metadata": {
            "kernelspec": {
                "display_name": "Python 3",
                "language": "python",
                "name": "python3"
            },
            "language_info": {
                "name": "python",
                "version": "3.8.0"
            }
        },
        "nbformat": 4,
        "nbformat_minor": 4
    }
    
    return notebook

# Create example notebook
notebook = create_session_notebook(sample_session_log)
print("Sample notebook structure:")
print(json.dumps(notebook, indent=2)[:500] + "...")

## Benefits of the Improvements

### 🔧 **Robust Port Management**
- No more "port already in use" errors when switching servers
- Automatic cleanup of stale SSH tunnels
- Dynamic port assignment for multiple concurrent sessions

### 📋 **Session Logging**
- Complete audit trail of session setup
- Easy troubleshooting with detailed logs
- Export capability for documentation

### 🔄 **Seamless Server Switching**
- Quick disconnection without resource leaks
- Preserved SSH connection for faster reconnection
- Automatic resource cleanup between sessions

### 🚀 **Auto-start Feature**
- Immediate Jupyter startup on server selection
- No manual button clicking required
- Real-time progress feedback

These improvements make the Jupyter launcher much more reliable and user-friendly for multi-server environments!

## Latest UI Improvements (Session 2)

### Fixed Issues:
1. **Port Display**: Fixed port label stuck on "Checking..." - now shows actual detected port
2. **Start Button**: Fixed button stuck on "Starting Jupyter..." - now shows "✅ Jupyter Running" when ready
3. **Auto-scroll**: Added automatic scrolling in terminal when new output appears

### Enhanced Browser Tab Detection:
- Better feedback when browser tabs are blocking ports
- Emoji indicators for different types of port usage:
  - 🌐 Browser tab detected
  - ⚙️ Other process using port
  - ✅ Port successfully freed
  - 🔍 Finding alternative port

In [ ]:
# Demonstrate the enhanced port detection functionality
import psutil
import socket

def demo_port_analysis(port=8888):
    """Demonstrate how the app now analyzes port usage."""
    
    print(f"🔍 Analyzing port {port}...")
    
    # Check if port is free
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.bind(('localhost', port))
            print(f"✅ Port {port} is available")
            return
    except OSError:
        print(f"⚠️  Port {port} is in use")
    
    # Analyze what's using the port
    browser_processes = ['chrome', 'firefox', 'edge', 'safari', 'msedge', 'chromium']
    
    try:
        for proc in psutil.process_iter(['pid', 'name', 'connections']):
            try:
                connections = proc.info.get('connections', [])
                for conn in connections:
                    if hasattr(conn, 'laddr') and conn.laddr and conn.laddr.port == port:
                        process_name = proc.info['name']
                        
                        if any(browser in process_name.lower() for browser in browser_processes):
                            print(f"🌐 Browser tab detected: {process_name} (PID: {proc.info['pid']})")
                            print("💡 Close the browser tab to free this port")
                        else:
                            print(f"⚙️  Process detected: {process_name} (PID: {proc.info['pid']})")
                        
                        return
            except (psutil.NoSuchProcess, psutil.AccessDenied):
                continue
    except Exception as e:
        print(f"❌ Error analyzing port: {e}")

# Test the port analysis
demo_port_analysis(8888)

### Auto-Scroll Terminal Feature

The terminal now automatically scrolls to the bottom when new output appears. This is implemented using a clientside callback:

```javascript
function(children) {
    if (children && children.length > 0) {
        setTimeout(function() {
            var terminalDiv = document.getElementById('terminal-output');
            if (terminalDiv) {
                terminalDiv.scrollTop = terminalDiv.scrollHeight;
            }
        }, 100);  // Small delay to ensure content is rendered
    }
    return window.dash_clientside.no_update;
}
```

**Benefits:**
- Users don't need to manually scroll to see new output
- Better user experience during long-running processes
- Immediate feedback when commands are sent